In [1]:
import numpy
import random
import statistics
import pandas as pd
import altair as alt

In [2]:
def reward(agents: list, i: int) -> int:
	return (reward(agents, i-1) + (agents[i] - agents[i-1])/(((agents[i] - statistics.quantiles(agents, n=5)[2])**2)/2 + len(agents) - i)) if i != 0 else agents[i]/(len(agents))

In [3]:
def generate_scores(min_=5, max_=40) -> list:
    """Generate a sample distribution with a random number of participating auditors between [min_, max_]"""
    auditors_score = [random.randint(1, 100) for _ in range(random.randint(min_, max_))]
    auditors_score.sort()
    if not 100 in auditors_score:
        auditors_score[-1] = 100 # Best score is assigned a perfect score for splitting the entire prize pool (else the share will amount to less than 100%)
    elif auditors_score.count(100) > 1:
        while auditors_score[-2] == 100: # Remove all other occurences of "perfect" score. Since list is sorted, the other occurences start to appear just before last index.
            del auditors_score[-2]
            
    return auditors_score

In [4]:
prize_pool = 2_000_000 # Set total prize_pool here

In [5]:
select_points = alt.selection_interval(encodings=['x'], empty='all') # Enables point selection brush on the graph (click+drag) 

In [6]:
unselected_color = '#DDDDDD'
mean_color = '#fa4d56'
selected_color = '#FC6471'

In [7]:
df = pd.DataFrame(generate_scores(200, 500), columns=['score'])
df['rewards'] = df.apply(lambda x: reward(list(df.score), list(df.score).index(x['score']))/100, axis=1)

base = alt.Chart(df).mark_circle(clip=True, size=45).encode(
    x=alt.X('score:Q', title="Score"),
    y=alt.Y('rewards:Q', title='Share of prize pool', axis=alt.Axis(format='%')),
    opacity=alt.value(1),
    color=alt.condition(
        select_points,
        alt.value(selected_color), alt.value(unselected_color)
    ),
    tooltip=['score:Q', 'rewards:Q']
)

polynomial_fit = [
    base.transform_filter(
        select_points
    ).transform_regression(
        "score", "rewards", method="poly", order=order, as_=["score", str(order)]
    )
    .mark_line()
    .transform_fold([str(order)], as_=["degree", "rewards"])
    .encode(alt.Color("degree:N", legend=alt.Legend(title="Regression order")), opacity=alt.value(.5))
    for order in [1, 3, 5]
]

score_distribution = alt.Chart(df).mark_bar(color=unselected_color).encode(
    x=alt.X('score:Q', title="Score distribution", bin=True),
    y=alt.Y('count()', title="")
).properties(width=800, height=100)

main = alt.layer(base, *polynomial_fit).resolve_scale(x='shared', y='shared').add_selection(
    select_points
).properties(width=800, height=400)

print(f"Total number of records:{len(df.index)}")
main & score_distribution

Total number of records:341


alt.VConcatChart(...)

In [8]:
cutoff = 50
smooth = 500

cutoff_exp = 30
smooth_exp = 10

df['value'] = (df['score'] - cutoff) / ((df['score'] - cutoff) ** 2 + smooth) ** (1/2)
df['value_exp'] = numpy.log(smooth_exp + numpy.exp(df['score'] - cutoff_exp))

alt.Chart(df).transform_calculate(
    v=(alt.datum.value - df['value'].min()) / (df['value'].max() - df['value'].min())
).mark_line(
    point=True
).encode(
    x='score:Q',
    y='v:Q'
) & alt.Chart(df).mark_line(
    point=True
).encode(
    x='score:Q',
    y='value_exp:Q'
)

alt.VConcatChart(...)